Import dependancies

In [14]:
import tensorflow as tf
import os
import numpy as np
import zipfile
import matplotlib.pyplot as plt
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator      #For rescale images
from tensorflow.python.keras.models import Sequential, Model
from __future__ import division, absolute_import, print_function, unicode_literals
from tensorflow.keras.utils import Sequence


# Make sure tenserflow version is above 2.0
try:
  %tensorflow_version 2.x
except Exvception:
  pass

Checking tensorflow version

In [2]:
tf.__version__

'2.2.0'

Download the flower dataset and unzip it

In [3]:
_URL = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"

zip_file = tf.keras.utils.get_file(origin=_URL, fname="flower_photos.tgz", extract=True)
base_dir = os.path.join(os.path.dirname(zip_file), 'flower_photos')

228818944/228813984 [==============================] - 4s 0us/step


In [5]:
img_size=224      #Image_height and image_size setting to this value
batch_size =64    #Batch size - Size of batches of data

1.   Rescale images using ImageDataGenerator()
2.   Create the train data generator and the validation data generator

In [6]:
#  Rescaling images to same using ImageDataGenerator

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

# Train data generator

train_gen = datagen.flow_from_directory(
    base_dir,     #train dataset directory
    target_size = (img_size,img_size),      #image size (setting height and width)
    batch_size=batch_size,      #size of batches of data
    subset='training'     #subset of data (training)
)

# Validation data generator
  # same procedure as train data generator

val_gen = datagen.flow_from_directory(
    base_dir,
    target_size = (img_size,img_size),
    batch_size=batch_size,
    subset='validation'     #subset of data (validation)
)


Found 2939 images belonging to 5 classes.
Found 731 images belonging to 5 classes.


In [7]:
# extracting labels and image features from train data 
for image_batch , label_batch in train_gen:
  break
  
print('Shepe of the image batch : ')
print(image_batch.shape)
print('Shepe of the label batch : ')
print(label_batch.shape)

Shepe of the image batch : 
(64, 224, 224, 3)
Shepe of the label batch : 
(64, 5)


In [8]:
print(train_gen.class_indices)      #checking class indices of train data

{'daisy': 0, 'dandelion': 1, 'roses': 2, 'sunflowers': 3, 'tulips': 4}


In [9]:
# saving the labels to a file, for later usage
labels = "\n".join(sorted(train_gen.class_indices.keys()))
with open('labels.txt','w') as f:
  f.write(labels)

In [10]:
!cat labels.txt     #view extracted labels in labels.txt

daisy
dandelion
roses
sunflowers
tulips

# Creating the base model with convnets

For creating the base model we have used MobileNet V2 model from google. also used pre-trained dataset "ImageNet"

Proforming "Feature Extraction"

Base layaer will be our immidiate layer for feature extraction
First, pick which intermediate layer of MobileNet V2 will be used for feature extraction. 

Excluding the top layer, classification layer is ideal.


In [11]:
IMG_SHAPE = (img_size, img_size, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                              include_top=False,      #excluding the classificaton layer
                                              weights='imagenet')

9412608/9406464 [==============================] - 1s 0us/step


# Feature extraction

Freeze the conv base created as base_model to use it as the feature extractor.
Then add classifer on top of it and train 

In [12]:
base_model.trainable = False

In [13]:
# Adding Classifrcation head
model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.Conv2D(32,3, activation ='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(5, activation = 'softmax')
])

# Compile the model

Compiling model before training.

In [15]:
model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss='categorical_crossentropy',      # loss function (because of many classes we have we used cateforical_crossentropy)
              metrics=['accuracy'])       # asking for accuracy metrix

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 7, 7, 1280)        2257984   
_________________________________________________________________
conv2d (Conv2D)              (None, 5, 5, 32)          368672    
_________________________________________________________________
dropout (Dropout)            (None, 5, 5, 32)          0         
_________________________________________________________________
global_average_pooling2d (Gl (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 5)                 165       
Total params: 2,626,821
Trainable params: 368,837
Non-trainable params: 2,257,984
_________________________________________________________________
